# 基于LSTM的字符生成

## LSTM

长短期记忆（long short-term memory，LSTM）是一种常用的门控循环神经网络。

LSTM 中引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞，从而记录额外的信息。

![LSTM](../Docs/LSTM-Core.png)

<center>http://dprogrammer.org/rnn-lstm-gru</center>

* 输入门、遗忘门、输出门计算

$$ \begin{aligned} 
\boldsymbol{I}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xi} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hi} + \boldsymbol{b}_i),\\ 
\boldsymbol{F}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xf} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hf} + \boldsymbol{b}f),\\ 
\boldsymbol{O}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xo} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{ho} + \boldsymbol{b}_o), \end{aligned} $$

* 候选记忆细胞计算

$$ \tilde{\boldsymbol{C}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xc} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hc} + \boldsymbol{b}_c), $$

* 更新记忆细胞
$$\boldsymbol{C}_t = \boldsymbol{F}_t \odot \boldsymbol{C}_{t-1} + \boldsymbol{I}_t \odot \tilde{\boldsymbol{C}}_t.$$

* 更新隐藏状态

$$\boldsymbol{H}_t = \boldsymbol{O}_t \odot \text{tanh}(\boldsymbol{C}_t).$$

In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import numpy as np
import pickle
import zipfile
import copy
import random

import utils as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## dataset

In [2]:
class TextTokenizer(object):
    def __init__(self, text=None, max_vocab=None, filename=None):
        if filename is not None:
            with open(filename, 'rb') as f:
                self.vocab = pickle.load(f)
        else:
            corpus_chars = set(text)
            # max_vocab_process
            vocab_count = {}
            for word in corpus_chars:
                if vocab_count.get(word) is None:
                    vocab_count[word] = 1
                else:
                    vocab_count[word] += 1

            vocab_count_list = []
            for word in vocab_count:
                vocab_count_list.append((word, vocab_count[word]))
            # sort count with number
            vocab_count_list.sort(key=lambda x: x[1], reverse=True)
            if max_vocab is not None and len(vocab_count_list) > max_vocab:
                vocab_count_list = vocab_count_list[:max_vocab]
            vocab = [x[0] for x in vocab_count_list]
            self.vocab = vocab

        self.char_to_index = {c: i for i, c in enumerate(self.vocab)}
        self.index_to_char = dict(enumerate(self.vocab))

    @property
    def vocab_size(self):
        return len(self.vocab) + 1

    def word_to_index(self, word):
        if word in self.char_to_index:
            return self.char_to_index[word]
        else:
            return len(self.vocab)

    def index_to_word(self, index):
        if index == len(self.vocab):
            return '<unk>'
        elif index < len(self.vocab):
            return self.char_to_index[index]
        else:
            raise Exception('Unknown index!')

    def text_to_array(self, text):
        arr = []
        for word in text:
            arr.append(self.word_to_index(word))
        return np.array(arr)

    def array_to_text(self, arr):
        words = []
        for index in arr:
            words.append(self.index_to_word(index))
        return "".join(words)

    def save_to_file(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.vocab, f)

In [3]:
# random sample
def data_loader_random(corpus_indices, seq_length, time_steps, device=None):
    num_samples = (len(corpus_indices) - 1) // time_steps
    epoch_size = num_samples // seq_length

    # shuffle sample
    samples_indices = list(range(num_samples))
    random.shuffle(samples_indices)

    # generator data
    for i in range(epoch_size):
        i = i * seq_length

        batch_incices = samples_indices[i: i + seq_length]
        x = [corpus_indices[indices: indices + time_steps] for indices in batch_incices]
        y = [corpus_indices[indices + 1: indices + time_steps + 1] for indices in batch_incices]

        x = torch.tensor(x, dtype=torch.float32).view(seq_length, time_steps)
        y = torch.tensor(y, dtype=torch.float32).view(seq_length, time_steps)

        yield x, y

In [4]:
# consecutive sample
def data_loader_consecutive(corpus_indices, seq_length, time_steps):

    data_len = len(corpus_indices)

    seq_size = data_len // seq_length

    # resize to => (seq_length, seq_size)
    corpus_indices = np.array(corpus_indices[: seq_size * seq_length], dtype=np.float).reshape((seq_length, -1))

    epoch_size = (seq_size - 1) // time_steps

    # generator data
    np.random.shuffle(corpus_indices)

    # convert to torch tensor
    torch_indices = torch.tensor(corpus_indices, dtype=torch.float32).view(seq_length, seq_size)
    for i in range(epoch_size):
        i = i * time_steps
        x = torch_indices[:, i: i + time_steps]
        y = torch_indices[:, i + 1: i + time_steps + 1]

        yield x, y

In [5]:
with zipfile.ZipFile('../Datasets/jaychou_lyrics/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')  # corpus

corpus_chars = corpus_chars.replace('\n', ' ').replace('\t', ' ')
corpus_chars = corpus_chars[: 20000]

tokenizer = TextTokenizer(text=corpus_chars, max_vocab=None)

vocab = tokenizer.vocab
char_to_index = tokenizer.char_to_index
index_to_char = tokenizer.index_to_char
vocab_size = tokenizer.vocab_size

corpus_indices = tokenizer.text_to_array(corpus_chars)

## model

In [6]:
def onehot(x, n_class):
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)

    return [F.one_hot(x[:, i].to(torch.int64), n_class).to(dtype=torch.float32, device=device) for i in range(x.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1448])


In [7]:
def init_lstm_state(seq_length, num_hiddens, device):
    return (torch.zeros((seq_length, num_hiddens), device=device),
            torch.zeros((seq_length, num_hiddens), device=device))

In [8]:
input_size, hidden_size, output_size = vocab_size, 256, vocab_size
# initial model params
def get_params():
    
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    def _three():
        return (_one((input_size, hidden_size)),
                _one((hidden_size, hidden_size)),
                nn.Parameter(torch.zeros(hidden_size, device=device, dtype=torch.float32),requires_grad=True))
    
    # hidden params
    w_xi, w_hi, b_i = _three() # input gate
    w_xf, w_hf, b_f = _three() # forget gate
    w_xo, w_ho, b_o = _three() # output gate

    w_xc, w_hc, b_c = _three()  # candidate memory 
    
    # output params
    w_hq = _one((hidden_size, output_size))
    b_q = nn.Parameter(torch.zeros(output_size, device=device, dtype=torch.float32), requires_grad=True)
    
    return nn.ParameterList([w_xi, w_hi, b_i, w_xf, w_hf, b_f, w_xo, w_ho, b_o, w_xc, w_hc, b_c, w_hq, b_q])

num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)


will use cuda


In [9]:
# lstm model
def lstm(inputs, states, params):

    """

    :param inputs: (time_steps, seq_len, vocab_size)
    :param states:
    :param params:
    :return:
    """

    (h, c) = states
    w_xi, w_hi, b_i, w_xf, w_hf, b_f, w_xo, w_ho, b_o, w_xc, w_hc, b_c, w_hq, b_q = params

    outputs = []

    for input in inputs:
        i_t = torch.sigmoid(torch.matmul(input, w_xi) + torch.matmul(h, w_hi) + b_i)
        f_t = torch.sigmoid(torch.matmul(input, w_xf) + torch.matmul(h, w_hf) + b_f)
        o_t = torch.sigmoid(torch.matmul(input, w_xo) + torch.matmul(h, w_ho) + b_o)

        c_tilde = torch.tanh(torch.matmul(input, w_xc) + torch.matmul(h, w_hc) + b_c)

        c = torch.mul(f_t, c) + torch.mul(i_t, c_tilde) # update carry state
        
        h = torch.mul(o_t, c.tanh()) # update hidden state
        
        y = torch.matmul(h, w_hq) + b_q
      
        outputs.append(y)
        
    return outputs, (h, c)

In [10]:
X = torch.arange(10).view(2, 5)
state = init_lstm_state(X.shape[0], hidden_size, device)
inputs = onehot(X.to(device), vocab_size)
print(inputs[0].size())
params = get_params()
outputs, state_new = lstm(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

torch.Size([2, 1448])
5 torch.Size([2, 1448]) torch.Size([2, 256])


In [11]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

In [12]:
def predict_lstm(prefix, num_chars, lstm, params, init_lstm_state, 
                 num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_lstm_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        
        # previous last outputs as current input
        x = onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # comuper outputs and state
        (y, state) = lstm(x, state, params)
        
        # convert output from index to char
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [13]:
p = predict_lstm('好想', 10, lstm, params, init_lstm_state, hidden_size, vocab_size,
                    device, index_to_char, char_to_index)

print(p)

好想军椅觉静忍威持占植蔓


In [14]:
def train_and_predict_lstm(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, seq_length, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_loader_random
    else:
        data_iter_fn = data_loader_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 
            state = init_lstm_state(seq_length, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, seq_length, num_steps)
        for X, Y in data_iter:
            X = X.to(device)
            Y = Y.to(device)
            if is_random_iter:  
                state = init_lstm_state(seq_length, num_hiddens, device)
            else:
                #
                for s in state:
                    s.detach_()

            inputs = onehot(X, vocab_size)
        
            (outputs, state) = lstm(inputs, state, params)
        
            outputs = torch.cat(outputs, dim=0)
        
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # cross entropy
            l = loss(outputs, y.long())

            # gradient clearing
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            # gradient clearing
            grad_clipping(params, clipping_theta, device) 
            
            d2l.sgd(params, lr, 1)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_lstm(prefix, pred_len, lstm, params, init_lstm_state,
                                        num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [15]:
num_epochs, time_steps, seq_length, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['好想', '不想']

In [16]:
# random sample
random_sample = True
train_and_predict_lstm(lstm, get_params, init_lstm_state, hidden_size,
                      vocab_size, device, corpus_indices, index_to_char,
                      char_to_index, random_sample, num_epochs, time_steps, lr,
                      clipping_theta, seq_length, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 1.070160, time 0.60 sec
 - 好想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的
 - 不想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔
epoch 100, perplexity 1.057306, time 0.58 sec
 - 好想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔
 - 不想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔
epoch 150, perplexity 1.054995, time 0.57 sec
 - 好想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔
 - 不想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔
epoch 200, perplexity 1.053482, time 0.59 sec
 - 好想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔
 - 不想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔
epoch 250, perplexity 1.053013, time 0.58 sec
 - 好想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让
 - 不想想著你 这样 甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让


In [17]:
# consecutive sample
random_sample = False
train_and_predict_lstm(lstm, get_params, init_lstm_state, hidden_size,
                      vocab_size, device, corpus_indices, index_to_char,
                      char_to_index, random_sample, num_epochs, time_steps, lr,
                      clipping_theta, seq_length, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 109.663761, time 0.57 sec
 - 好想 我不要你不要 你不要我想要 我不要我不要 你不不再爱你 我不要我不要 你不不再爱你 我不要我不要 
 - 不想 我不要你不要 你不要我想要 我不要我不要 你不不再爱你 我不要我不要 你不不再爱你 我不要我不要 
epoch 100, perplexity 11.054566, time 0.57 sec
 - 好想 我不想这想 你不是陪不不开 我不要这样你 你永是我不要 你永远 你不是我不要 你不去 你不么我不多
 - 不想 你不么这不打 我不了这样活 我该会不生 你的手不是 你不了好离 我永远 你走了离开 我知好好生活 
epoch 150, perplexity 2.629018, time 0.57 sec
 - 好想要你 我要不想 我不不再 我不能 我不我 不想这样 你不么 你不开不开 不知不觉 你跟经这节奏 后知
 - 不想要你的手 想要到你去去 我只到到你已离开 我不要这样己  说是你不要 不要帮你不多说不要我带不难 我
epoch 200, perplexity 1.417030, time 0.62 sec
 - 好想要多场的剧 从想连无都都有不去 坐馨的黑乐喜上 周杰来的旧 这一种味道 做人的美丽 是风完著日 这里
 - 不想 这你为么简你 一直忙的话作 我的天都都的你 想想了直去的勇 我想着打慢的想脸 我不揍你已经很久 别
epoch 250, perplexity 1.167032, time 0.57 sec
 - 好想要一直场 我想你 这手对听 没有它着地暗乐下愿 看必让让点里方祷记 一直一阵以你 你经忆美不到 我想
 - 不想要 我只想想你的我 不要再觉汉活 是什么我说分开 我想 我只会陪下着你手不放你 爱可可可以简单单没没
